In [1]:
!pip install openpyxl pandas transformers accelerate bitsandbytes deep-translator

In [1]:
import pandas as pd

df = pd.read_excel('Traits List.xlsx')
essential_traits = [
    'Active', 'Adventurous', 'Affectionate', 'Ambitious', 'Angry', 'Anxious', 'Arrogant',
    'Attentive', 'Bossy', 'Brave', 'Calm', 'Capable', 'Careful', 'Cautious', 'Charismatic',
    'Cheerful', 'Clever', 'Clumsy', 'Cold-Hearted', 'Compassionate', 'Confident', 'Considerate',
    'Cooperative', 'Courageous', 'Cowardly', 'Critical', 'Cruel', 'Curious', 'Daring',
    'Decisive', 'Dependable', 'Determined', 'Diligent', 'Dishonest', 'Disrespectful',
    'Eager', 'Easygoing', 'Efficient', 'Eloquent', 'Embarrassed', 'Energetic', 'Enthusiastic',
    'Fair', 'Faithful', 'Fearless', 'Friendly', 'Funny', 'Generous', 'Gentle', 'Grateful',
    'Greedy', 'Grouchy', 'Gullible', 'Happy', 'Helpful', 'Honest', 'Hopeful', 'Humble',
    'Impulsive', 'Independent', 'Innocent', 'Intelligent', 'Jealous', 'Kind', 'Lazy',
    'Logical', 'Lonely', 'Loving', 'Loyal', 'Mean', 'Mature', 'Moody', 'Naïve', 'Nervous',
    'Obedient', 'Optimistic', 'Organized', 'Pessimistic', 'Polite', 'Popular', 'Positive',
    'Proud', 'Reliable', 'Respectful', 'Responsible', 'Rude', 'Sarcastic', 'Selfish',
    'Sensitive', 'Shy', 'Smart', 'Sociable', 'Stubborn', 'Sweet', 'Talkative', 'Thoughtful',
    'Timid', 'Trustworthy', 'Wise', 'Witty'
]

df = df[df['Trait'].isin(essential_traits)].reset_index(drop=True)
df.head()

,Trait,Description
0,Active,Tending to move around often; full of energy
1,Adventurous,Willing to take risks; likes new adventures & ...
2,Affectionate,Showing affection; tender; loving
3,Ambitious,Eager to succeed; full of desire
4,Angry,Feeling or showing anger


In [2]:
df.shape

(91, 2)

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from deep_translator import GoogleTranslator
import torch
import re

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
system_prompt_template = """
Generate a system prompt that instructs an LLM to produce responses that exemplify the following personality trait. The system prompt should guide the model’s response style, tone, and content to reflect the trait’s characteristics. Use this format exactly:

Trait: {trait}
Description: {description}

Respond in this format exactly:

System Prompt: You are [Original trait with strength adjective for describing this trait]. Your responses should [detailed instructions on how to embody the trait].
Sentiment: [Positive/Negative/Neutral]

Only respond in the format shown above, without additional text, explanations, or repetition.
"""

def translate_trait(trait):
    ru = GoogleTranslator(source="en", target="ru").translate(trait)
    uk = GoogleTranslator(source="en", target="uk").translate(trait)
    return ru, uk

def generate_answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")
    output = model.generate(**inputs, max_new_tokens=250, do_sample=True, temperature=0.1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

def parse_response(text):
    match = re.search(
        r"System Prompt:\s*You are (.*?)\. Your responses should (.*?)\.?\s*Sentiment:\s*(Positive|Negative|Neutral)",
        text,
        flags=re.DOTALL
    )
    matches = re.findall(
        r"System Prompt:\s*You are (.*?)\. Your responses should (.*?)\.?\s*Sentiment:\s*(Positive|Negative|Neutral)",
        text,
        flags=re.DOTALL
    )
    if not matches:
        return None
    trait, behavior, sentiment = matches[-1]
    return {
        "System Prompt": f"You are {trait}. Your responses should {behavior.strip()}.",
        "Sentiment": sentiment
    }


responses = []

for _, row in df.iterrows():
    trait = row['Trait']
    description = row['Description']
    prompt = system_prompt_template.format(trait=trait, description=description)
    raw_response = generate_answer(prompt)
    cleaned_response = raw_response.replace(prompt.strip(), '').strip()
    russian, ukrainian = translate_trait(trait)
    parsed_response = parse_response(cleaned_response)
    print(cleaned_response)
    responses.append({
        "Trait": trait,
        "System Prompt": parsed_response["System Prompt"],
        "Russian": russian,
        "Ukrainian": ukrainian,
        "Sentiment": parsed_response["Sentiment"]
    })

df_prompts = pd.DataFrame(responses)
df_prompts.shape

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


System Prompt: You are Active. Your responses should be full of energy and movement. Use vivid language to describe your actions and experiences. Sentiment: Positive.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


System Prompt: You are adventurous. Your responses should be willing to take risks and embrace new adventures and experiences. Sentiment: Positive.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


System Prompt: You are trustworthy. Your responses should be able to trust, dependable, and responsible.
Sentiment: Neutral


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


System Prompt: You are Wise. Your responses should be thoughtful, insightful, and reflective. Use your knowledge and experience to provide guidance and advice to those seeking it. Sentiment: Positive.
System Prompt: You are witty. Your responses should be clever and amusing. Sentiment: Positive.


(91, 5)

In [31]:
df_prompts

,Trait,System Prompt,Russian,Ukrainian,Sentiment
0,Active,You are Active. Your responses should be full ...,Активный,Активний,Positive
1,Adventurous,You are adventurous. Your responses should be ...,Авантюрный,Авантюрний,Positive
2,Affectionate,You are Affectionate. Your responses should be...,Любящий,Ласкавий,Positive
3,Ambitious,You are Ambitious. Your responses should be fu...,Амбициозный,Амбітний,Positive
4,Angry,"You are Angry. Your responses should be loud, ...",Злой,Розлючений,Negative
...,...,...,...,...,...
86,Thoughtful,You are thoughtful. Your responses should trea...,Вдумчивый,Задумливий,Positive
87,Timid,You are timid. Your responses should show a la...,Робкий,Боязкий,Negative
88,Trustworthy,You are trustworthy. Your responses should be ...,Заслуживает доверия,Надійний,Neutral
89,Wise,You are Wise. Your responses should be thought...,Мудрый,Мудрий,Positive


In [32]:
df_prompts.to_csv("traits_with_prompts.csv", index=False)

In [33]:
import requests
import json
import random
random.seed(42)

url = "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json"
response = requests.get(url)
alpaca_df = response.json()

N = 100

final_dataset = []
for _, row in df_prompts.iterrows():
    trait = row["Trait"]
    system_prompt = row["System Prompt"]
    samples = random.sample(alpaca_df, N)

    for sample in samples:
        instruction = sample["instruction"]
        input_text = sample.get("input", "").strip()

        if input_text:
            user_prompt = f"{instruction}\n\nInput: {input_text}"
        else:
            user_prompt = instruction

        full_prompt = f"<|system|>\n{system_prompt}\n<|user|>\n{user_prompt}\n<|assistant|>\n"
        entry = {
            "trait": trait,
            "system_prompt": system_prompt,
            "alpaca_instruction": instruction,
            "alpaca_input": input_text,
            "full_prompt": full_prompt
        }
        final_dataset.append(entry)

with open("trait_combined_dataset.json", "w", encoding="utf-8") as f:
    json.dump(final_dataset, f, ensure_ascii=False, indent=2)

final_dataset[0]

{'trait': 'Active',
 'system_prompt': 'You are Active. Your responses should be full of energy and movement. Use vivid language to describe your actions and experiences.',
 'alpaca_instruction': 'What are the advantages of using a Scrum Agile methodology?',
 'alpaca_input': '',
 'full_prompt': '<|system|>\nYou are Active. Your responses should be full of energy and movement. Use vivid language to describe your actions and experiences.\n<|user|>\nWhat are the advantages of using a Scrum Agile methodology?\n<|assistant|>\n'}